### Fine Tuning Bert

In [1]:
from datasets import load_dataset, ClassLabel
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments
from peft import LoraConfig, TaskType, PeftModel, get_peft_model
from pathlib import Path
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

mdl_tok_name = "distilbert-base-uncased"

C:\Users\vtsoumpris\PycharmProjects\Lightweight_FineTuning_FoundationModel\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Lora Configuration for Bert model

1. **Target Modules is set to:**
    - `q_lin` (query), `k_lin` (key), and `v_lin` (value): The fundamental layers in the attention mechanism crucial for capturing relationships in the inputs.
    - `out_lin`: This layer outputs the processed attention values, making it an important target for parameter-efficient fine-tuning as well.

By including all four (`q_lin`, `k_lin`, `v_lin`, and `out_lin`), I can ensure full coverage of attention-related parameters while keeping the number of learnable parameters minimal due to low-rank adaptation (`r=8`).

In [2]:
lora_config = \
	LoraConfig(
		r = 8,  # Low-rank dimension: Start with 8 for a compact model like DistilBERT
		target_modules = ["q_lin", "k_lin", "v_lin", "out_lin"],  # Correct target modules for attention layers in DistilBERT
		task_type = TaskType.SEQ_CLS,  # Task type, e.g., Sequence Classification
		lora_alpha = 16,  # Scaling factor: smaller due to the lightweight architecture
		lora_dropout = 0.1  # Dropout, increase slightly to prevent overfitting on smaller models
	)

#### Loading the filtered dataset

In [3]:
# Define the file path to the dataset
file_path = Path("data/filtered_dataset.csv")

# Load the dataset using Hugging Face's `load_dataset`
dataset = load_dataset('csv', data_files = str(file_path))

# Inspect the unique values in the 'labels' column
product_classes = dataset["train"].unique("Product")

# Convert the 'Product' column to a ClassLabel feature
product_label = ClassLabel(names=product_classes)
dataset = dataset.cast_column("Product", product_label)

# Rename the columns: "Product" to "labels", and "Consumer complaint narrative" to "complaint"
dataset = dataset.rename_column("Product", "labels")
dataset = dataset.rename_column("Consumer complaint narrative", "complaint")

# Extract the features (columns) we want
dataset = \
    dataset["train"].select_columns(
        ["complaint", "labels"]
    ).train_test_split(
        test_size=0.2,
        shuffle=True,
        seed=23,
        stratify_by_column="labels"
    )

splits = ["train", "test"]

# View the resulting dataset
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['complaint', 'labels'],
        num_rows: 800
    })
    test: Dataset({
        features: ['complaint', 'labels'],
        num_rows: 200
    })
})


#### Inspecting the labels

Credit card is labeled as 0 and  Mortgage is labeled as 1

In [4]:
product_label

ClassLabel(names=['Mortgage', 'Credit card or prepaid card'], id=None)

#### Preprocess dataset

Tokenizing 'Consumer complaint narrative' feature values

In [5]:
tokenizer = AutoTokenizer.from_pretrained(mdl_tok_name)

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["complaint"],
                            truncation=True,
                            padding=True,
                            return_tensors = "pt"
                            ),
	    batched=True,

    )


# Inspect the available columns in the dataset
tokenized_dataset["train"]

Map: 100%|██████████| 200/200 [00:00<00:00, 1851.88 examples/s]


Dataset({
    features: ['complaint', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 800
})

#### Loading Model

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(
    mdl_tok_name,
    num_labels=2,
    id2label={0: "Mortgage", 1: "Credit card or prepaid card"},
    label2id={"Credit card or prepaid card": 0, "Mortgage": 1},
)
print(model)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [7]:
# If you added new tokens, resize the model's embeddings accordingly
#model.resize_token_embeddings(len(tokenizer))

In [8]:
peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

trainable params: 887,042 || all params: 67,842,052 || trainable%: 1.3075


#### Defining Evaluation Metrics as a function

In [9]:
def compute_metrics(eval_pred):
	# Unpack predictions and labels
	predictions, labels = eval_pred
	# Get the predicted class (argmax selects the class with the highest score)
	predictions = np.argmax(predictions, axis = 1)
	# Compute metrics
	accuracy = accuracy_score(y_true = labels, y_pred = predictions)
	precision = precision_score(y_true =labels, y_pred =predictions)
	recall = recall_score(y_true = labels, y_pred = predictions)
	f1 = f1_score(y_true = labels, y_pred = predictions)
	# Return all metrics
	return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


#### Define Trainer to fine-tuning the foundation model

The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.

You can find more at this [link](https://huggingface.co/docs/transformers/main_classes/trainer).

In [10]:
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir= "./data/creditc_mortg",
        # Learning rate
        learning_rate= 2e-5,  #2e-5
        # Train/Validate batch size
        per_device_train_batch_size= 4,  #16 # Reduce batch size to avoid memory crashes
        per_device_eval_batch_size= 4, #16 # Same for evaluation
        # Evaluate and save the model after each epoch
        eval_strategy= "epoch", # Evaluate at the end of each epoch
        save_strategy= "epoch", # Save model checkpoint every epoch
	    # Epochs and weight decay
        num_train_epochs= 1, # Start with 1 epoch, increase as needed
        weight_decay= 0.01,  #Standard weight decay
	    # Resource management
		gradient_accumulation_steps= 4,  # Simulate larger batches with accumulation
	    #
        load_best_model_at_end= True,
	    use_cpu= True, # Ensure no GPU usage
    ),
    train_dataset= tokenized_dataset["train"],
    eval_dataset= tokenized_dataset["test"],
    tokenizer= tokenizer,
    data_collator= DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics= compute_metrics,
)

C:\Users\vtsoumpris\AppData\Local\Temp\ipykernel_15268\3076374690.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


#### Start fine-tuning

In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.680192,0.725000,0.710280,0.760000,0.734300


TrainOutput(global_step=50, training_loss=0.6887635040283203, metrics={'train_runtime': 615.9872, 'train_samples_per_second': 1.299, 'train_steps_per_second': 0.081, 'total_flos': 108153913344000.0, 'train_loss': 0.6887635040283203, 'epoch': 1.0})

#### Validate fine-tuned model

In [12]:
trainer.evaluate()

{'eval_loss': 0.6801918148994446,
 'eval_accuracy': 0.725,
 'eval_precision': 0.7102803738317757,
 'eval_recall': 0.76,
 'eval_f1': 0.7342995169082126,
 'eval_runtime': 51.9587,
 'eval_samples_per_second': 3.849,
 'eval_steps_per_second': 0.962,
 'epoch': 1.0}

In [13]:
peft_model.save_pretrained("./vtsoumpris/fnc-distilbert-lora")

In [15]:
# Make a dataframe with the predictions and the text and the labels
items_for_manual_review = tokenized_dataset["test"].select(
    [0, 1, 22, 31, 43, 199, 150, 40]
)

results = trainer.predict(items_for_manual_review)
df = pd.DataFrame(
    {
        "complaint": [item["complaint"] for item in items_for_manual_review],
        "predictions": results.predictions.argmax(axis=1),
        "labels": results.label_ids,
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df

,complaint,predictions,labels
0,"I reported fraudulent activity on my visa credit card with Bank America. There were two fraudulent charges made on XX/XX/XXXX and XX/XX/XXXX at the same location in amount of {$39.00}. I filed a claim with Bank of America and they denied the claim with no explanation other than saying I have too many accounts with this pizza place, which makes no sense. I contacted them again and they said they'd have to file an appeal it would be another 45-90 days. In the meantime while I'm waiting for appeal, they added the two charges back into my account which is continue to accrue interest. Please help me with these fraudulent charges and unfair treatment from Bank of America",1,1
1,"I had decided to cancel my homeowners insurance with one company, and go with another company 2 months prior to the end date of the policy. I had gone to a local branch to inform of the change, and make sure the company that I was leaving would not be paid and the new one would be paid for the upcoming year renewal. The representative took down all the information, and stated that she would make sure the now old insurance company would n't be paid, and would only release a payment to the new insurance company. I received a notice this week to inform that both insurance companies have been paid in a total amount over {$2400.00}. After paying two different insurance company Bank of America ran an out-of-cycle escrow analysis report knowing that two insurance companies had been paid ; which will cause my mortgage payment to increase effective XX/XX/XXXX, 2018 close to {$200.00}. I have contacted the branch representative who stated that she had forgot to get the information over to the mortgage company, and it was my responsibility to contact the insurance company to get the refund back on a cancelled policy. I was informed by the insurance company that a refund could take up to 7 weeks. In the meantime, the mortgage company knowing that an error was made on there end will not remove the escrow analysis which will cause the mortgage payment to increase.",0,0
2,"There are already multiple complaints that Bank of America ignored. Their regulatory complaint staff refuse to return phone calls. On XXXX XXXX, XXXX, I got scammed and thought I was speaking to Bank of America corporate. Unfortunately, it was a criminal enterprise unknown to me when I was trying to verify my information with the crook to get help. As the fraud was happening, I contacted Bank of America right when the fraud was occurring ( the thief without my knowledge or consent obtained cash advances on 4 of my credit cards and somehow sent the money out on one of my checking accounts. ) The fraud dept sadistically insisted that as a woman age XXXX and high risk for covid go into a branch and refused to speak to me. No one was allowed to talk to me. No fraud forms were mailed to me asking for an explanation and a signature which proves they weren't doing anything. Finally, I lucked out with the TN call center and the total freeze was removed. I was able to close my demand deposit accounts. Previous complaints were ignored and their regulatory complaints rep refuses to call me back. Due to the compassion of one fraud person who made an appointment, I did go into the XXXX XXXX and XXXX XXXX was masked and we were in his office. ( A very nasty and sadistic creature named XXXX at the XXXX fraud center stooped so low she took away my online banking when I called when I saw a {$2000.00} fraud credit card balance. ) At that time, only my card ending in XXXX hat was reissued and Iused, had a legitimate balance of {$200.00} ) XXXX showed me that the other credit cards had 0 balances. I sat in his office for an hour while he was on hold with the fraud dept, I left and he continued to hold for them for another hour. XXXX called me -his number is XXXX and advised me that the fraud claims were settled per his conversation ). I believe XXXX but he was given incorrect information. Sa